<a href="https://colab.research.google.com/github/LordJonny1000/Beispielsatzgenerator/blob/main/Bonus_Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bonus Exercises 2: Low-Rank Adaptation and Crosslingual Transfer**



This notebook represents the second bonus exercises for the lecture Multilingual and Crosslingual Methods and Language Resources (2024W 340168-1). For each successfully completed bonus exercise, a maximum of three points can be achieved that will be added to the points of the final exam. The tasks to be completed in the following notebook are marked with 👋 ⚒.

---




In this notebook, we will use Low-Rank Adaptation to Fine-Tune XLM-R on the task of linguistic acceptability in English and then test its zero-shot capability in other languages.

# **Make sure to set your runtime to GPU before you start training.**

(Tab: Runtime/Change Runtime Type -> Select GPU)

-----------
## **Fine-Tuning on English**

The first part has already been prepared for you. We will load and preprocess the Corpus for Linguistic Acceptability (COLA) dataset from GLUE and then use Low-Rank Adaptation to fine-tune XLM-R.

### Installation

As always, we first need to install the necessary libraries. One that is new in this notebook is the Parameter-Efficient Fine-Tuning (PEFT) library.

In [1]:
!pip install -U evaluate
!pip install -U datasets
!pip install -U transformers
!pip install -U peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━

### Loading the Dataset

In this notebook we will first be using the COLA dataset from the GLUE library and then a multilingual extension.
 We will first train on English and transfer to another language and evaluate zero-shot transfer on one more language (see [here](https://huggingface.co/datasets/Geralt-Targaryen/MELA) for a selection).

In [2]:
from datasets import load_dataset

dataset_en = load_dataset("glue", "cola")
dataset_en.num_rows

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/251k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/37.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

{'train': 8551, 'validation': 1043, 'test': 1063}

Let us take a look at the components of the dataset.

In [7]:
dataset_en['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['unacceptable', 'acceptable'], id=None),
 'idx': Value(dtype='int32', id=None)}

Hugging Face Datasets is designed to be interoperable with libraries like Pandas, as well as NumPy, PyTorch, TensorFlow, and JAX. To enable the conversion between various third-party libraries, Hugging Face Datasets provides a Dataset.set_format() function. This function only changes the output format of the dataset, so you can easily switch to another format without affecting the underlying data format which is Apache Arrow. The formatting is done in-place, so let’s convert our dataset to Pandas and look at a random sample:

In [4]:
from IPython.display import display, HTML

dataset_en.set_format("pandas")
df = dataset_en["train"][:]
# Create a random sample
sample = df.sample(n=5, random_state=42)
display(HTML(sample.to_html()))

,sentence,label,idx
2389,Angela characterized Shelly as a lifesaver.,1,2389
5048,They're not finding it a stress being in the same office.,1,5048
3133,Paul exhaled on Mary.,0,3133
5955,I ordered if John drink his beer.,0,5955
625,Press the stamp against the pad completely.,1,625


The Pandas dataframe can now be used as we would always use Pandas, for instance to count the number of labels for `cause` in the column question.

In [5]:
df["label"].value_counts()

,count
label,
1,6023
0,2528


We can see that the two labels are spread quite evenly across the two types of questions.

This was just a brief detour to show how datasets can be nicely manipulated and displayed using other libraries. We will now get back to our usual datasets library from Hugging Face. To this end, we will reset the format.

In [6]:
dataset_en.reset_format()

### Preprocessing the Dataset

In this example, we model COPA as a multiple-choice task with two choices. Thus, we encode the premise and question as well as both choices as one input to our `xlm-roberta-base` model. Using `dataset.map()`, we can pass the full dataset through the tokenizer in batches.

In [8]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")
batch_size = 32

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding=True, truncation=True)

def preprocess_dataset(dataset):
  token_dataset = dataset.map(tokenize_function, batched=True, batch_size=batch_size)
  tokenized_dataset = token_dataset.rename_column("label", "labels")
  return tokenized_dataset

data_set_en_with_test = DatasetDict(
    train=dataset_en['train'].shuffle(seed=24).select(range(7488)),
    validation=dataset_en['validation'],
    test=dataset_en['train'].shuffle(seed=24).select(range(7488, 8551)),
)

tokenized_dataset_en = preprocess_dataset(data_set_en_with_test)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/7488 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [9]:
tokenized_dataset_en["train"][1]

{'sentence': 'John knows that she left and whether she will come back.',
 'labels': 1,
 'idx': 7246,
 'input_ids': [0,
  4939,
  93002,
  450,
  2412,
  25737,
  136,
  36766,
  2412,
  1221,
  1380,
  4420,
  5,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

-----------
## **Low-Rank Adaptation (LoRA)**

In order to perform low-rank adaptation (LoRA) on a pretrained language model for parameter-efficient fine-tuning (PEFT), we need to set a few parameters in the LoRA Configuration. Hugging Face offers some [documentation on LoRA](https://huggingface.co/docs/peft/main/en/developer_guides/lora).

The `task-type` specifies which task the model should be fine-tuned on and needs to correspond to the way the model is loaded. If we load a model for Sequence Classification, also the task needs to be `SEQ_CLS`, an abbreviation for Sequence Classification. Then the dataset needs to be one with an input sequence and a number of target classes.

The `target-module` depends on the type of model, which for XLM-R is `["query", "value"]`. Since we wish to change model parameters, the inference mode is set to false. The variable `r`indicates the rank to which the dimensionality is being reduced. The variable `alpha` is a scaling parameter, because `r`scales at 1.0. With small datasets or if unsure, the rank and alpha can be the same. Finally, dropout is a random omission of trainable parameters (setting to zero) during training, mostly to avoid overfitting.

Feel free to play with and adapt these parameters if you are interested in seeing the effect.


In [10]:
from peft import LoraConfig, PeftType, get_peft_model
from transformers import AutoModelForSequenceClassification

peft_type = PeftType.LORA
peft_config = LoraConfig(task_type="SEQ_CLS", target_modules=["query", "value"], inference_mode=False, r=8, lora_alpha=8, lora_dropout=0.1)
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-large")
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,838,082 || all params: 561,730,564 || trainable%: 0.3272


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): XLMRobertaForSequenceClassification(
      (roberta): XLMRobertaModel(
        (embeddings): XLMRobertaEmbeddings(
          (word_embeddings): Embedding(250002, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): XLMRobertaEncoder(
          (layer): ModuleList(
            (0-23): 24 x XLMRobertaLayer(
              (attention): XLMRobertaAttention(
                (self): XLMRobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
     

In [11]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer, EvalPrediction
from datasets import concatenate_datasets

num_train_epochs = 5
logging_steps = len(tokenized_dataset_en["train"]) // (batch_size * num_train_epochs)
accuracy = evaluate.load("accuracy")

training_args = TrainingArguments(
    learning_rate=2e-4,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=logging_steps,
    output_dir="./training_output",
    overwrite_output_dir=True,
    report_to='none',
    load_best_model_at_end=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_en["train"],
    eval_dataset=tokenized_dataset_en["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Once we have configured the model with PEFT, we can train the PEFT model as usual.

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.593500,0.588314,0.693193
2,0.545900,0.520775,0.750719
3,0.506900,0.522151,0.772771
4,0.455700,0.539208,0.785235
5,0.412300,0.522610,0.792905


TrainOutput(global_step=1170, training_loss=0.5231680079403087, metrics={'train_runtime': 572.9628, 'train_samples_per_second': 65.345, 'train_steps_per_second': 2.042, 'total_flos': 2688914965511424.0, 'train_loss': 0.5231680079403087, 'epoch': 5.0})

/content
['checkpoint-936', 'checkpoint-234', 'checkpoint-1170', 'checkpoint-702', 'checkpoint-468']


👋 ⚒ Evaluate on the English test set to see how well the fine-tuning has worked.

In [31]:
# Your code for the evaluation here
import torch
from peft import PeftModel, PeftConfig

peft_model_id = "/content/training_output/checkpoint-1170"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
inference_model = PeftModel.from_pretrained(inference_model, peft_model_id)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [91]:
from tqdm import tqdm


dataset = tokenized_dataset_en["test"]
# for x in dataset:
#   print(dataset[x])

predictions = []
labels = dataset['labels']
for sent in tqdm(dataset['sentence']):
    model_inputs = tokenizer(sent, return_tensors="pt")
    outputs = inference_model(**model_inputs)
    prediction = int(outputs.logits.argmax(dim=-1))
    predictions.append(prediction)

success = []
for prediction, label in tqdm(zip(predictions, labels)):
  success.append(int(prediction == label))


print(f"Acc.: {sum(success)/len(success)}")




100%|██████████| 1063/1063 [08:06<00:00,  2.18it/s]
1063it [00:00, 1032787.85it/s]

Acc.: 0.7958607714016933


## **Crosslingual Transfer**

In this section, we will be using the Multilingual Evaluation of Linguistic Acceptability ([MELA](https://github.com/sjtu-compling/mela?tab=readme-ov-file)), which is also [available on Hugging Face](https://huggingface.co/datasets/Geralt-Targaryen/MELA) to test the transfer and zero-shot capabilities of XLM-R with LoRA Fine-Tuning.

We will first fine-tune on German and then test the on German but also in a zero-shot approach on another language of your choice.

Please be aware of the fact that MELA "only" offers a dev and a test set - no train, validation, test split. Thus, the preprocessing needs to be slightly adapted.

In [92]:
from datasets import load_dataset

de = load_dataset("Geralt-Targaryen/MELA", "de")
dataset_de = preprocess_dataset(de)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/945 [00:00<?, ? examples/s]

{'idx': 'c1-1.1_n6-b', 'labels': 0, 'sentence': 'Heute, die Maus hat den Käse verschmäht', 'input_ids': [0, 67203, 4, 68, 23788, 7, 1256, 168, 23734, 184, 35607, 10065, 9703, 2, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]}


👋 ⚒ Use the German dev partition to further-finetune the previously configured model and then evaluate on the test partition of the German dataset.

In [ ]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

# Fine-tuning on German dev set here
tokenized_datasets = dataset_de["dev"].map(tokenize_function, batched=True)
training_args = TrainingArguments(output_dir="fine_tuned", eval_strategy="epoch")




trainer = Trainer(

    model=inference_model,

    args=training_args,

    train_dataset=dataset_de["dev"].map(tokenize_function, batched=True),

    eval_dataset=dataset_de["test"].map(tokenize_function, batched=True),

    compute_metrics=compute_metrics,

)


trainer.train()



Map:   0%|          | 0/945 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

👋 ⚒ Select another language of your choice from the [MELA dataset](https://huggingface.co/datasets/Geralt-Targaryen/MELA) to only evaluate the fine-tuned model (zero-shot capability).

**Alternative**: Feel free to create your own mini-dataset of a few (non)-acceptable sentences in a language of your choice to test the model's zero-shot capacity.

In [ ]:
# Your import evaluate

import evaluate

metric = evaluate.load("accuracy")

model.eval()

for batch in eval_dataloader:

    batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():

        outputs = model(**batch)

    logits = outputs.logits

    predictions = torch.argmax(logits, dim=-1)

    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()